# Kaggle House Price Prob.



2018.10.20.Sat. By Taeu

for Google Machine learning study

*******************************************************

# Content
## 1. Data Skimming

    1-1. Data collect
    1-2. Data load
    1-3. Data Skimming ( to excel _ )


## 2. Data Preprocessing

    2-1. Fill NAs
    2-2. Drop
    2-3. Editing ( Adding, Editing, Binning etc..)
    
    
## 3. Model & Evaluation

    3-1 pipeline
    3-2 Model Selection & Eval
    3-3 Result - Submission
    
    
*******************************************************

In [1]:
# import 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
plt.style.use('ggplot')

from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline, make_pipeline
from scipy.stats import skew
from sklearn.decomposition import PCA, KernelPCA
from sklearn.preprocessing import Imputer

from sklearn.model_selection import cross_val_score, GridSearchCV, KFold
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import SVR, LinearSVR
from sklearn.linear_model import ElasticNet, SGDRegressor, BayesianRidge
from sklearn.kernel_ridge import KernelRidge
#from xgboost import XGBRegressor
# if you want to use XGBoost
'''
# install xgboost
http://cleancode-ws.tistory.com/79
# understanding xgboost
https://www.slideshare.net/freepsw/boosting-bagging-vs-boosting
'''

'\n# install xgboost\nhttp://cleancode-ws.tistory.com/79\n# understanding xgboost\nhttps://www.slideshare.net/freepsw/boosting-bagging-vs-boosting\n'

## 1. Data Skimming

# 1-2. data load

In [2]:
train = pd.read_csv('input/train.csv')
test = pd.read_csv('input/test.csv')

In [3]:
full = pd.concat([train,test],ignore_index=True)

# 1-3. data skimming

In [4]:
cols = train.columns
nulls = full.isnull().sum()
nulls = nulls[nulls>0]
nulls_index = nulls.index

In [5]:
datacols = []
datanull = []
datatype = []
datavalue = []

for col in cols:
    datacols += [col]
    datatype += [train[col].dtype]
    datavalue+= [train[col][0]]
    if col in nulls_index :
        datanull += [nulls[col]]
    else :
        datanull += [0]
        
idd = list(range(cols.size))

In [14]:
# to make checklist of cols(features)
pd.DataFrame({'num': idd, 'columns': datacols, 'datanull':datanull ,'datavalue':datavalue,
              'datatype': datatype }).to_csv('hp_columns_info.csv', index =False)

## 2. Data Preprocessing

### 2-1. Missing Value

In [6]:
full.groupby(['Neighborhood'])[['LotFrontage']].agg(['mean','median','count'])

full['LotAreaCut']= pd.qcut(full.LotArea,10)

full.groupby(['LotAreaCut'])[['LotFrontage']].agg(['mean','median','count'])

LotFrontage             
                           mean median count
LotAreaCut                                  
(1299.999, 4922.4]    35.741036   34.0   251
(4922.4, 7007.6]      55.460674   52.0   267
(7007.6, 7960.4]      63.008000   62.0   250
(7960.4, 8741.0]      66.964844   65.0   256
(8741.0, 9453.0]      70.106996   70.0   243
(9453.0, 10151.6]     73.972656   75.0   256
(10151.6, 11001.2]    73.650794   75.0   252
(11001.2, 12203.8]    83.377193   82.0   228
(12203.8, 14300.6]    84.991228   85.0   228
(14300.6, 215245.0]   94.188119   90.0   202

In [7]:
full['LotFrontage'] = full.groupby(['LotAreaCut','Neighborhood'])['LotFrontage'].transform(lambda x : x.fillna(x.median()))
full['LotFrontage'].isnull().sum()

9

In [8]:
# 나머지 변수들은 다 0 이나 None 값으로
# nulls_index = nulls.index

for col in nulls_index:
    na_value = 0
    if full[col].dtype == object :
        na_value = "None"
    full[col].fillna(na_value, inplace = True)

In [9]:
# check it's done
a = full.isnull().sum()
a[a>0]

Series([], dtype: int64)

### 2-2. drop 

In [10]:
drop_index = ['SalePrice','Id']
for var in drop_index:
    full.drop([var],axis=1,inplace = True)

### 2-3. Feature Engineering


#### (1) type 변경

In [11]:
full_col = full.columns
for var in full_col:
    if full[var].dtype == object :
        full[var] = full[var].astype(str)

In [12]:
time = ['GarageYrBlt','YearBuilt','YearRemodAdd','MoSold','YrSold']
categorical = ['MSSubClass','OverallQual','OverallCond',]
strtype = time + categorical
for var in strtype:
    full[var] = full[var].astype(str)

In [13]:
type(full['LotAreaCut'][0])
full['LotAreaCut'] = full['LotAreaCut'].astype(str)
full.drop('LotAreaCut',axis=1,inplace=True)


In [23]:
# float 변경은 scale 조정에서 float로 바뀌므로 생략
full_col = full.columns

#### (2) scale 조정

In [24]:
# numeric 변수들 모음
numeric_col = []
for var in full_col:
    if type(full[var][0]) != str:
        numeric_col += [var]
    

In [25]:
# http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
# ~.fit(data), ~.fit_transform()

minmax = MinMaxScaler()
standard = StandardScaler()
robust = RobustScaler()

In [26]:
# 각 scale을 따로 적용하기 위해서 full의 copy version을 만든다.
full1 = full.copy()
full2 = full.copy()
full3 = full.copy()
full4 = full.copy()

In [27]:
#full1 은 no scale
full2[numeric_col] = minmax.fit_transform(full2[numeric_col])
full3[numeric_col] = standard.fit_transform(full3[numeric_col])
full4[numeric_col] = robust.fit_transform(full4[numeric_col])

In [28]:
full1[numeric_col].head(3) # full1, full2, full3, full4 들을 확인해보면 다 다른 값들을 가지고 있음
#full1은 numeric형을 float로 바꿔줄 필요가있음.

,1stFlrSF,2ndFlrSF,3SsnPorch,BedroomAbvGr,BsmtFinSF1,BsmtFinSF2,BsmtFullBath,BsmtHalfBath,BsmtUnfSF,EnclosedPorch,...,LotFrontage,LowQualFinSF,MasVnrArea,MiscVal,OpenPorchSF,PoolArea,ScreenPorch,TotRmsAbvGrd,TotalBsmtSF,WoodDeckSF
0,856,854,0,3,706.0,0.0,1.0,0.0,150.0,0,...,65.0,0,196.0,0,61,0,0,8,856.0,0
1,1262,0,0,3,978.0,0.0,0.0,1.0,284.0,0,...,80.0,0,0.0,0,0,0,0,6,1262.0,298
2,920,866,0,3,486.0,0.0,1.0,0.0,434.0,0,...,68.0,0,162.0,0,42,0,0,6,920.0,0


In [29]:
for var in numeric_col:
    full1[var] = full1[var].astype(float)

#### (3) categorical 변수 조정

In [115]:
aa = pd.concat([train,test],ignore_index=True)

In [116]:
aa.groupby(['MSSubClass'])[['SalePrice']].agg(['mean','median','count'])

SalePrice                
                     mean    median count
MSSubClass                               
20          185224.811567  159250.0   536
30           95829.724638   99900.0    69
40          156125.000000  142500.0     4
45          108591.666667  107500.0    12
50          143302.972222  132000.0   144
60          239948.501672  215200.0   299
70          166772.416667  156000.0    60
75          192437.500000  163500.0    16
80          169736.551724  166500.0    58
85          147810.000000  140750.0    20
90          133541.076923  135980.0    52
120         200779.080460  192000.0    87
150                   NaN       NaN     0
160         138647.380952  146000.0    63
180         102300.000000   88500.0    10
190         129613.333333  128250.0    30

In [30]:
full2["oMSSubClass"] = full2.MSSubClass.map({'180':1, 
                                    '30':2, '45':2, 
                                    '190':3, '50':3, '90':3, 
                                    '85':4, '40':4, '160':4, 
                                    '70':5, '20':5, '75':5, '80':5, '150':5,
                                    '120': 6, '60':6})
full3["oMSSubClass"] = full3.MSSubClass.map({'180':1, 
                                    '30':2, '45':2, 
                                    '190':3, '50':3, '90':3, 
                                    '85':4, '40':4, '160':4, 
                                    '70':5, '20':5, '75':5, '80':5, '150':5,
                                    '120': 6, '60':6})
full4["oMSSubClass"] = full4.MSSubClass.map({'180':1, 
                                    '30':2, '45':2, 
                                    '190':3, '50':3, '90':3, 
                                    '85':4, '40':4, '160':4, 
                                    '70':5, '20':5, '75':5, '80':5, '150':5,
                                    '120': 6, '60':6})

# 해주고 scale 변환을 위해 위의 scale 코드 재실행

In [ ]:
# map 함수 설정
def map_values(full):
    full["oMSSubClass"] = full.MSSubClass.map({'180':1, 
                                        '30':2, '45':2, 
                                        '190':3, '50':3, '90':3, 
                                        '85':4, '40':4, '160':4, 
                                        '70':5, '20':5, '75':5, '80':5, '150':5,
                                        '120': 6, '60':6})
    
    full["oMSZoning"] = full.MSZoning.map({'C (all)':1, 'RH':2, 'RM':2, 'RL':3, 'FV':4})
    
    full["oNeighborhood"] = full.Neighborhood.map({'MeadowV':1,
                                               'IDOTRR':2, 'BrDale':2,
                                               'OldTown':3, 'Edwards':3, 'BrkSide':3,
                                               'Sawyer':4, 'Blueste':4, 'SWISU':4, 'NAmes':4,
                                               'NPkVill':5, 'Mitchel':5,
                                               'SawyerW':6, 'Gilbert':6, 'NWAmes':6,
                                               'Blmngtn':7, 'CollgCr':7, 'ClearCr':7, 'Crawfor':7,
                                               'Veenker':8, 'Somerst':8, 'Timber':8,
                                               'StoneBr':9,
                                               'NoRidge':10, 'NridgHt':10})
    
    full["oCondition1"] = full.Condition1.map({'Artery':1,
                                           'Feedr':2, 'RRAe':2,
                                           'Norm':3, 'RRAn':3,
                                           'PosN':4, 'RRNe':4,
                                           'PosA':5 ,'RRNn':5})
    
    full["oBldgType"] = full.BldgType.map({'2fmCon':1, 'Duplex':1, 'Twnhs':1, '1Fam':2, 'TwnhsE':2})
    
    full["oHouseStyle"] = full.HouseStyle.map({'1.5Unf':1, 
                                           '1.5Fin':2, '2.5Unf':2, 'SFoyer':2, 
                                           '1Story':3, 'SLvl':3,
                                           '2Story':4, '2.5Fin':4})
    
    full["oExterior1st"] = full.Exterior1st.map({'BrkComm':1,
                                             'AsphShn':2, 'CBlock':2, 'AsbShng':2,
                                             'WdShing':3, 'Wd Sdng':3, 'MetalSd':3, 'Stucco':3, 'HdBoard':3,
                                             'BrkFace':4, 'Plywood':4,
                                             'VinylSd':5,
                                             'CemntBd':6,
                                             'Stone':7, 'ImStucc':7})
    
    full["oMasVnrType"] = full.MasVnrType.map({'BrkCmn':1, 'None':1, 'BrkFace':2, 'Stone':3})
    
    full["oExterQual"] = full.ExterQual.map({'Fa':1, 'TA':2, 'Gd':3, 'Ex':4})
    
    full["oFoundation"] = full.Foundation.map({'Slab':1, 
                                           'BrkTil':2, 'CBlock':2, 'Stone':2,
                                           'Wood':3, 'PConc':4})
    
    full["oBsmtQual"] = full.BsmtQual.map({'Fa':2, 'None':1, 'TA':3, 'Gd':4, 'Ex':5})
    
    full["oBsmtExposure"] = full.BsmtExposure.map({'None':1, 'No':2, 'Av':3, 'Mn':3, 'Gd':4})
    
    full["oHeating"] = full.Heating.map({'Floor':1, 'Grav':1, 'Wall':2, 'OthW':3, 'GasW':4, 'GasA':5})
    
    full["oHeatingQC"] = full.HeatingQC.map({'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5})
    
    full["oKitchenQual"] = full.KitchenQual.map({'Fa':1, 'TA':2, 'Gd':3, 'Ex':4})
    
    full["oFunctional"] = full.Functional.map({'Maj2':1, 'Maj1':2, 'Min1':2, 'Min2':2, 'Mod':2, 'Sev':2, 'Typ':3})
    
    full["oFireplaceQu"] = full.FireplaceQu.map({'None':1, 'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5})
    
    full["oGarageType"] = full.GarageType.map({'CarPort':1, 'None':1,
                                           'Detchd':2,
                                           '2Types':3, 'Basment':3,
                                           'Attchd':4, 'BuiltIn':5})
    
    full["oGarageFinish"] = full.GarageFinish.map({'None':1, 'Unf':2, 'RFn':3, 'Fin':4})
    
    full["oPavedDrive"] = full.PavedDrive.map({'N':1, 'P':2, 'Y':3})
    
    full["oSaleType"] = full.SaleType.map({'COD':1, 'ConLD':1, 'ConLI':1, 'ConLw':1, 'Oth':1, 'WD':1,
                                       'CWD':2, 'Con':3, 'New':3})
    
    full["oSaleCondition"] = full.SaleCondition.map({'AdjLand':1, 'Abnorml':2, 'Alloca':2, 'Family':2, 'Normal':3, 'Partial':4})            
                
                        
                        
    
    return full

## 3. Model & Evaluation

### 3-1. Pipeline

In [31]:
# string -> one_hot_encoding
full1 = pd.get_dummies(full1)
full2 = pd.get_dummies(full2)
full3 = pd.get_dummies(full3)
full4 = pd.get_dummies(full4)

In [32]:
full3.shape

(2919, 638)

In [33]:
# y 값
# label 분리 SalePrice 부분, 그리고 log scale까지
y = train['SalePrice']
y_log = np.log(y)

In [34]:
# x 값들 (columns, features, inputs)
n_train = train.shape[0]
x1 = full1[:n_train]
x2 = full2[:n_train]
x3 = full3[:n_train]
x4 = full4[:n_train]

test_x1 = full1[n_train:]
test_x2 = full2[n_train:]
test_x3 = full3[n_train:]
test_x4 = full4[n_train:]

### 3-2. Model & Eval

In [35]:
def rmse_cv(model,X,y):
    rmse = np.sqrt(-cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv=5))
    return rmse

In [36]:
models = [LinearRegression(),
          Ridge(),
          Lasso(alpha=0.01,max_iter=10000),
          RandomForestRegressor(),
          GradientBoostingRegressor(),
          SVR(),
          LinearSVR(),
          ElasticNet(alpha=0.001,max_iter=10000),
          SGDRegressor(max_iter=1000,tol=1e-3),
          BayesianRidge(),
          KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5),
          ExtraTreesRegressor(),
          ]

# if you have xgboost package, put XGBRegressor() in models 

In [73]:
names = ["LR", "Ridge", "Lasso", "RF", "GBR", "SVR", "LinSVR", "Ela","SGD","Bay","Ker","Extra"]
for name, model in zip(names, models):
    score = rmse_cv(model, x1 , y_log)
    print("{}: {:.6f}, {:.4f}".format(name,score.mean(),score.std()))

LR: 5695.002996, 10104.9169
Ridge: 0.154480, 0.0211
Lasso: 0.190063, 0.0370
RF: 0.159517, 0.0109
GBR: 0.136248, 0.0113
SVR: 0.397849, 0.0162
LinSVR: 5.705883, 2.3262
Ela: 0.137034, 0.0252
SGD: 94861304249887952.000000, 37373237047621120.0000
Bay: 0.142505, 0.0219
Ker: 2.243725, 1.3767
Extra: 0.162765, 0.0157


In [37]:
names = ["LR", "Ridge", "Lasso", "RF", "GBR", "SVR", "LinSVR", "Ela","SGD","Bay","Ker","Extra"]
for name, model in zip(names, models):
    score = rmse_cv(model, x2 , y_log)
    print("{}: {:.6f}, {:.4f}".format(name,score.mean(),score.std()))

LR: 2395746080.863004, 2650053927.6431
Ridge: 0.153850, 0.0187
Lasso: 0.209864, 0.0166
RF: 0.160690, 0.0099
GBR: 0.135205, 0.0118
SVR: 0.163891, 0.0122
LinSVR: 0.174963, 0.0221
Ela: 0.134248, 0.0175
SGD: 0.240759, 0.0157
Bay: 0.145871, 0.0157
Ker: 0.160164, 0.0122
Extra: 0.164468, 0.0146


In [38]:
names = ["LR", "Ridge", "Lasso", "RF", "GBR", "SVR", "LinSVR", "Ela","SGD","Bay","Ker","Extra"]
for name, model in zip(names, models):
    score = rmse_cv(model, x3 , y_log)
    print("{}: {:.6f}, {:.4f}".format(name,score.mean(),score.std()))

LR: 1572655774.586774, 1654773878.7078
Ridge: 0.155112, 0.0222
Lasso: 0.180672, 0.0239
RF: 0.157593, 0.0112
GBR: 0.136095, 0.0120
SVR: 0.134181, 0.0122
LinSVR: 0.177948, 0.0266
Ela: 0.137374, 0.0250
SGD: 0.266193, 0.0164
Bay: 0.142825, 0.0221
Ker: 0.138970, 0.0194
Extra: 0.162734, 0.0193


In [39]:
names = ["LR", "Ridge", "Lasso", "RF", "GBR", "SVR", "LinSVR", "Ela","SGD","Bay","Ker","Extra"]
for name, model in zip(names, models):
    score = rmse_cv(model, x4 , y_log)
    print("{}: {:.6f}, {:.4f}".format(name,score.mean(),score.std()))

LR: 6118849.805382, 10756533.1355
Ridge: 0.155072, 0.0222
Lasso: 0.183248, 0.0249
RF: 0.157090, 0.0112
GBR: 0.135944, 0.0129
SVR: 0.223618, 0.0179
LinSVR: 0.612395, 0.4610
Ela: 0.137483, 0.0249
SGD: 440932584788897.562500, 522404381266385.8125
Bay: 0.142807, 0.0220
Ker: 0.382613, 0.3316
Extra: 0.164096, 0.0166


In [40]:
## Grid Search
class grid():
    def __init__(self,model):
        self.model = model
    
    def grid_get(self,X,y,param_grid):
        grid_search = GridSearchCV(self.model,param_grid,cv=5, scoring="neg_mean_squared_error")
        grid_search.fit(X,y)
        print(grid_search.best_params_, np.sqrt(-grid_search.best_score_))
        grid_search.cv_results_['mean_test_score'] = np.sqrt(-grid_search.cv_results_['mean_test_score'])
        print(pd.DataFrame(grid_search.cv_results_)[['params','mean_test_score','std_test_score']])

## Lasso

In [41]:
grid(Lasso()).grid_get(x2,y_log,{'alpha': [0.0004,0.0005,0.0007,0.0009],'max_iter':[10000]})
grid(Lasso()).grid_get(x3,y_log,{'alpha': [0.0004,0.0005,0.0007,0.0009],'max_iter':[10000]})
grid(Lasso()).grid_get(x4,y_log,{'alpha': [0.0004,0.0005,0.0007,0.0009],'max_iter':[10000]})

{'alpha': 0.0007, 'max_iter': 10000} 0.13505469399196174
                                 params  mean_test_score  std_test_score
0  {'alpha': 0.0004, 'max_iter': 10000}         0.136359        0.005717
1  {'alpha': 0.0005, 'max_iter': 10000}         0.135366        0.005329
2  {'alpha': 0.0007, 'max_iter': 10000}         0.135055        0.004776
3  {'alpha': 0.0009, 'max_iter': 10000}         0.136180        0.004533
{'alpha': 0.0007, 'max_iter': 10000} 0.13923735739691173
                                 params  mean_test_score  std_test_score
0  {'alpha': 0.0004, 'max_iter': 10000}         0.140456        0.007740
1  {'alpha': 0.0005, 'max_iter': 10000}         0.139771        0.007699
2  {'alpha': 0.0007, 'max_iter': 10000}         0.139237        0.007598
3  {'alpha': 0.0009, 'max_iter': 10000}         0.140025        0.007606
{'alpha': 0.0007, 'max_iter': 10000} 0.13940379280237872
                                 params  mean_test_score  std_test_score
0  {'alpha': 0.0004, 'max_

## Ridge

In [42]:
grid(Ridge()).grid_get(x2,y_log,{'alpha':[35,40,45,50,55,60,65,70,80,90]})
grid(Ridge()).grid_get(x3,y_log,{'alpha':[35,40,45,50,55,60,65,70,80,90]})
grid(Ridge()).grid_get(x4,y_log,{'alpha':[35,40,45,50,55,60,65,70,80,90]})

{'alpha': 35} 0.15065237072784315
          params  mean_test_score  std_test_score
0  {'alpha': 35}         0.150652        0.003750
1  {'alpha': 40}         0.152161        0.003777
2  {'alpha': 45}         0.153621        0.003808
3  {'alpha': 50}         0.155029        0.003842
4  {'alpha': 55}         0.156384        0.003876
5  {'alpha': 60}         0.157689        0.003912
6  {'alpha': 65}         0.158946        0.003948
7  {'alpha': 70}         0.160156        0.003984
8  {'alpha': 80}         0.162451        0.004055
9  {'alpha': 90}         0.164593        0.004126
{'alpha': 35} 0.14504820825045484
          params  mean_test_score  std_test_score
0  {'alpha': 35}         0.145048        0.007109
1  {'alpha': 40}         0.145501        0.007168
2  {'alpha': 45}         0.145965        0.007219
3  {'alpha': 50}         0.146431        0.007264
4  {'alpha': 55}         0.146891        0.007303
5  {'alpha': 60}         0.147343        0.007338
6  {'alpha': 65}         0.14778

## ElasticNet

In [43]:
grid(ElasticNet()).grid_get(x2,y_log,{'alpha':[0.0008,0.004,0.005],'l1_ratio':[0.08,0.1,0.3],'max_iter':[10000]})
grid(ElasticNet()).grid_get(x3,y_log,{'alpha':[0.0008,0.004,0.005],'l1_ratio':[0.08,0.1,0.3],'max_iter':[10000]})
grid(ElasticNet()).grid_get(x4,y_log,{'alpha':[0.0008,0.004,0.005],'l1_ratio':[0.08,0.1,0.3],'max_iter':[10000]})

{'alpha': 0.004, 'l1_ratio': 0.1, 'max_iter': 10000} 0.13765284550807125
                                              params  mean_test_score  \
0  {'alpha': 0.0008, 'l1_ratio': 0.08, 'max_iter'...         0.147435   
1  {'alpha': 0.0008, 'l1_ratio': 0.1, 'max_iter':...         0.146071   
2  {'alpha': 0.0008, 'l1_ratio': 0.3, 'max_iter':...         0.138672   
3  {'alpha': 0.004, 'l1_ratio': 0.08, 'max_iter':...         0.137996   
4  {'alpha': 0.004, 'l1_ratio': 0.1, 'max_iter': ...         0.137653   
5  {'alpha': 0.004, 'l1_ratio': 0.3, 'max_iter': ...         0.142358   
6  {'alpha': 0.005, 'l1_ratio': 0.08, 'max_iter':...         0.138285   
7  {'alpha': 0.005, 'l1_ratio': 0.1, 'max_iter': ...         0.138384   
8  {'alpha': 0.005, 'l1_ratio': 0.3, 'max_iter': ...         0.146460   

   std_test_score  
0        0.005753  
1        0.005710  
2        0.005333  
3        0.004036  
4        0.003965  
5        0.003830  
6        0.003865  
7        0.003804  
8        0.00391

## Kernel Ridge

In [44]:
param_grid={'alpha':[0.2,0.3,0.4], 'kernel':["polynomial"], 'degree':[3],'coef0':[0.8,1]}
grid(KernelRidge()).grid_get(x2,y_log,param_grid)
grid(KernelRidge()).grid_get(x3,y_log,param_grid)
grid(KernelRidge()).grid_get(x4,y_log,param_grid)

{'alpha': 0.2, 'coef0': 1, 'degree': 3, 'kernel': 'polynomial'} 0.15051501225923827
                                              params  mean_test_score  \
0  {'alpha': 0.2, 'coef0': 0.8, 'degree': 3, 'ker...         0.156816   
1  {'alpha': 0.2, 'coef0': 1, 'degree': 3, 'kerne...         0.150515   
2  {'alpha': 0.3, 'coef0': 0.8, 'degree': 3, 'ker...         0.164699   
3  {'alpha': 0.3, 'coef0': 1, 'degree': 3, 'kerne...         0.156094   
4  {'alpha': 0.4, 'coef0': 0.8, 'degree': 3, 'ker...         0.171700   
5  {'alpha': 0.4, 'coef0': 1, 'degree': 3, 'kerne...         0.161120   

   std_test_score  
0        0.003413  
1        0.003440  
2        0.003498  
3        0.003490  
4        0.003591  
5        0.003557  
{'alpha': 0.2, 'coef0': 1, 'degree': 3, 'kernel': 'polynomial'} 0.12801432117629127
                                              params  mean_test_score  \
0  {'alpha': 0.2, 'coef0': 0.8, 'degree': 3, 'ker...         0.132891   
1  {'alpha': 0.2, 'coef0': 1, 'deg

In [45]:
class AverageWeight(BaseEstimator, RegressorMixin):
    def __init__(self,mod,weight):
        self.mod = mod
        self.weight = weight
        
    def fit(self,X,y):
        self.models_ = [clone(x) for x in self.mod]
        for model in self.models_:
            model.fit(X,y)
        return self
    
    def predict(self,X):
        w = list()
        pred = np.array([model.predict(X) for model in self.models_])
        # for every data point, single model prediction times weight, then add them together
        for data in range(pred.shape[1]):
            single = [pred[model,data]*weight for model,weight in zip(range(pred.shape[0]),self.weight)]
            w.append(np.sum(single))
        return w

lasso = Lasso(alpha=0.0005,max_iter=10000)
ridge = Ridge(alpha=35)
svr = SVR(gamma= 0.0004,kernel='rbf',C=13,epsilon=0.009)
ker = KernelRidge(alpha=0.2 ,kernel='polynomial',degree=3 , coef0=0.8)
ela = ElasticNet(alpha=0.005,l1_ratio=0.08,max_iter=10000)
bay = BayesianRidge()

# assign weights based on their gridsearch score
w1 = 0.2
w2 = 0.1
w3 = 0.0
w4 = 0.5
w5 = 0.2
w6 = 0.0

weight_avg = AverageWeight(mod = [lasso,ridge,svr,ker,ela,bay],weight=[w1,w2,w3,w4,w5,w6])


score = rmse_cv(weight_avg,x3,y_log) #커널만 x3쓰기 pred에서 지금이 제일 낮은 item 찾음
print(score.mean())

0.1313671368441154


## stacking

## 3.3 Result - Submission

In [136]:
# 여러모델 평균
r1 = lasso.fit(x3, y_log)
r2 = ridge.fit(x3,y_log)
r3 = svr.fit(x3,y_log)
r4 = ker.fit(x3,y_log)
r5 = ela.fit(x3,y_log)
r6 = bay.fit(x3,y_log)

pred_r1 =np.exp(r1.predict(test_x3))
pred_r2 =np.exp(r2.predict(test_x3))
pred_r3 =np.exp(r3.predict(test_x3))
pred_r4 =np.exp(r4.predict(test_x3))
pred_r5 =np.exp(r5.predict(test_x3))
pred_r6 =np.exp(r6.predict(test_x3))

pred_final = w1 * pred_r1 + w2 * pred_r2 + w3 * pred_r3 + w4 * pred_r4 + w5 * pred_r5 + w6 * pred_r6


result=pd.DataFrame({'Id':test.Id, 'SalePrice':pred_final})
result.to_csv("submission5.csv",index=False)

In [137]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url = "https://user-images.githubusercontent.com/24144491/47170710-adfa5800-d341-11e8-8063-fe1312148e83.png")


In [92]:
Image(url = "https://user-images.githubusercontent.com/24144491/47187882-c3d34180-d370-11e8-9456-a86cc62615c0.PNG")

In [139]:
# categorical 변수 mapping 해서 numeric으로 넣어주니까 스코어는 더 낮아짐.
Image(url = "https://user-images.githubusercontent.com/24144491/47188599-ace21e80-d373-11e8-8799-7e0f289dd706.PNG")


In [140]:
# x2, x3 교차해서 결과내니 성능 더 낮아짐
Image(url = "https://user-images.githubusercontent.com/24144491/47188508-4c52e180-d373-11e8-8d9c-38f3c4b9d910.PNG")


In [141]:
## 추가로 해야할 것들, mapping 한 결과들 한 번 다 넣어보기. 그리고 lasso, 다른 결과들이 어떻게 내는지 확인해볼 것. 